Класс линейной регрессии обучающийся градиентным спуском

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

Веса обновляются следующим образом:

𝑤(𝑡+1)=𝑤(𝑡)−𝛼𝑡∇𝑄(𝑤(𝑡)), 

а  𝑗 -ая компонента градиента расписывается как

∂𝑄(𝑤)𝑤𝑗=2𝑙∑𝑖=1𝑙𝑥𝑖𝑗(⟨𝑤,𝑥𝑖⟩−𝑦𝑖).

Реализация градиентного спуска

In [ ]:
from sklearn.base import BaseEstimator

class LinearRegression(BaseEstimator):
    # функция инициализации
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    # функция обучения линейной модели 
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """

        # количество объектов и признаков
        l, d = X.shape

        # если не задано начальное приближение, то иницализация веса нулями
        if self.w0 is None:
          self.w0 = np.zeros(d)

        # итоговый вес, будет равен w0
        self.w = self.w0

        # итерации градиентного спуска по количеству шагов max_steps
        for step in range(self.max_steps):
          # записываем в историю текущие веса
          self.w_history.append(self.w)
        
          # новые веса w_new = w - шаг обучения * градиент (веса обновляем по формуле выше)
          w_new = self.w - self.alpha * self.calc_gradient(X, y)

          # если веса не сильно различаются, то делаем следующие шаги градиентного спуска
          # считаем разность между новыми весами и старыми w_new - self.w
          # расчет нормы от это разницы (np.linalg.norm(w_new - self.w)
          # если норма меньше чем epsilon, то выходим из цикла
          if (np.linalg.norm(w_new - self.w) < self.epsilon):
            break
          
          # обновляем веса (записывам новые веса)
          self.w = w_new

        # возвращаем сам класс
        return self
    
    # метод, который по новым объектам будет возвращать предсказание модели
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """

        # если нет весов, то возвращаем, что модель не обучена
        if self.w is None:
            raise Exception('Not trained yet')

        l, d = X.shape

        # лист с результатами
        y_pred = []

        for i in range(l):
          # применение линейной модели - скалярное произведение X[i] на w для конкретного объекта
          y_pred.append(np.dot(X[i], self.w))

        return np.array(y_pred)

    # служебная функция для расчета градиента в текущей точке
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d) матрица объектов и признаки
        y: np.array (l) целевая переменная
        ---
        output: np.array (d)
        """

        l, d = X.shape
        # лист с градиентами
        gradient = []

        # итерация по всем признакам, для получения конкретной компоненты j
        for j in range(d):
          # сумма равна нулю
          dQ = 0
          # итерация по всем объектам
          for i in range(l):
            # скалярное произведение векторов весов и X[i]: np.dot(X[i], self.w)
            dQ += (2/l) * X[i][j] * (np.dot(X[i], self.w) - y[i])
          gradient.append(dQ)

        return np.array(gradient)